In [ ]:
# import required libraries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

import json
from pathlib import Path
from types import SimpleNamespace

In [ ]:
# creating datasets from tfrecords
TRAIN_DIR = "/kaggle/input/tfrecords20k4/tfrecords20k4"
VALIDATION_DIR = "/kaggle/input/tfrecords-val-new/tfrecords20kvalnew"

train_builder = tfds.builder_from_directory(TRAIN_DIR)
validation_builder = tfds.builder_from_directory(VALIDATION_DIR)

In [ ]:
# Converting an image to tensor
image = tf.keras.preprocessing.image.load_img(path)
img_tensor = tf.keras.preprocessing.image.img_to_array(image, dtype="uint8")
img_tensor = tf.image.resize(img_tensor, (224, 224))
tf.cast(img_tensor, tf.uint8)

In [ ]:
# Loading metadata file
metadata = pd.read_csv(path, sep="\t")
new_metadata = metadata.copy()
# new_metadata["dataset_name"] = /.name
# return new_metadata

In [ ]:
# Extracting all Categories
categories = metadata["Category"].unique().tolist()

In [ ]:
# Extracting all ingredients
unique_ingredients = set()
for ingredient_list in metadata["Ingredients"]:
    ingredient_list = ingredient_list.split(",")
    unique_ingredients.update(ingredient_list)

In [ ]:
# Generate tensors for the images
def get_tensors(self, index):
        img_tensor = load_image_to_arr(img_path)
        if img_path.suffix == ".jpeg" or img_path.suffix == ".jpg":
            img_tensor = tf.io.encode_jpeg(img_tensor, format="rgb")
        elif img_path.suffix == ".png":
            img_tensor = tf.io.encode_png(img_tensor)
           
        calorie_tensor = row["Calorie(kcal)"]
        carbs_tensor = row["Carbohydrate(g)"]
        protein_tensor = row["Protein(g)"]
        fat_tensor = row["Fat(g)"]
        return img_tensor, {
            "category_output": tf.constant(row["Category"]),
            "calorie_output": tf.constant(calorie_tensor),
            "carbs_output": tf.constant(carbs_tensor),
            "protein_output": tf.constant(protein_tensor),
            "fat_output": tf.constant(fat_tensor),
            "ingredients_output": tf.constant(row["Ingredients"]),
        }


In [ ]:
def flatten_tensors(self, tensor):
        result = []
        img_data = tensor[0].numpy()
        others_data = [value.numpy() for key, value in tensor[1].items()]
        result.append(img_data)
        result.extend(others_data)
        return result

In [ ]:
def get_file_data(index, dataset_index):
    target_dataset = DATASETS[dataset_index]
    return target_dataset.flatten_tensors(target_dataset.get_tensors(index))

In [ ]:
def build_data_pipeline(datasets, sample_size=None):
    if sample_size is None:
        sample_size = [1.0] * len(datasets)
    assert len(sample_size) == len(
        datasets
    ), "Illegal array of sample sizes provided. Number of sample size does not match number of datasets"
    file_pointers = [
        x.extract_file_pointers().sample(frac=s, random_state=999)
        for x, s in zip(datasets, sample_size)
    ]
    all_file_pointers = pd.concat(file_pointers).sample(frac=1, random_state=999)
    print(f"Total samples : {len(all_file_pointers)}")
    
    all_file_pointers["dataset_name"] = all_file_pointers["dataset_name"].apply(
        lambda x: DATASETS_NAME.index(x)
    )

    final_dataset = tf.data.Dataset.from_tensor_slices(
        (
            all_file_pointers["metadata_index"].tolist(),
            all_file_pointers["dataset_name"].tolist(),
        )
    )
    return final_dataset

In [ ]:
 def get_category_one_hot_encoding(self, category_name):
        index = self.all_food_categories_integer_encoded[category_name]
        assert index is not None, f"{category_name} does not have an integer mapping"
        num_classes = len(self.all_food_categories)
        return keras.utils.to_categorical(index, num_classes, dtype="uint8")

In [ ]:
def get_ingredients_one_hot_encoding(self, ingredient_list):
        ingredient_list = list(
            map(lambda x: self.__transform_ingredient_to_integer(x), ingredient_list)
        )
        multi_one_hot_layer = tf.keras.layers.CategoryEncoding(
            num_tokens=len(self.all_ingredients), output_mode="multi_hot"
        )
        return tf.cast(multi_one_hot_layer(ingredient_list), dtype=tf.uint8)

In [ ]:
 def __transform_ingredient_to_integer(self, ingredient_name):
        index = self.all_ingredients_integer_encoded[ingredient_name]
        assert index is not None, f"{ingredient_name} does not have an integer mapping"
        return index

In [ ]:
def __encode_categories_to_integers(self):
        return {
            category_name: index
            for index, category_name in enumerate(self.all_food_categories)
        }

In [ ]:
def __encode_ingredients_to_integers(self):
        return {
            ingredient_name: index
            for index, ingredient_name in enumerate(self.all_ingredients)
        }

In [ ]:
import pandas as pd
FOOD101 = Food101(
    image_dir="data/images",
    metadata_dir="data/metadata",
)

DATASETS = [FOOD101]
DATASETS_NAME = [x.name for x in DATASETS]

def create_one_hot_encoder(datasets):
    all_categories = []
    all_ingredients = []
    for x in datasets:
        all_categories.extend(x.all_categories)
        all_ingredients.extend(x.all_ingredients)
    all_categories = set(all_categories)
    all_ingredients = set(all_ingredients)
    return OneHotEncoder([*all_categories], [*all_ingredients])

In [ ]:
(
    train_dataset_train,
    validation_dataset_train,
) = food101_builder.as_dataset(split=["train[:70%]", "train[70%:]"])

train_dataset = train_dataset_train.concatenate(validation_dataset_train)

(
    train_dataset_val,
    validation_dataset_val,
) = food101_val_builder.as_dataset(split=["train[:70%]", "train[70%:]"])

validation_dataset = train_dataset_val.concatenate(validation_dataset_val)

In [ ]:
print(f"Total validation size : {train_dataset.cardinality().numpy()}")
print(f"Total validation size : {validation_dataset.cardinality().numpy()}")

In [ ]:
BATCH_SIZE = 32
train_dataset = (
    train_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)
validation_dataset = (
    validation_dataset.map(parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
)

In [ ]:
def compile_model(self, optimizer):
        assert self.model is not None
        category_classification_loss = keras.losses.CategoricalCrossentropy()
        calorie_regression_loss = keras.losses.MeanAbsoluteError()
        carbs_regression_loss = keras.losses.MeanAbsoluteError()
        protein_regression_loss = keras.losses.MeanAbsoluteError()
        fat_regression_loss = keras.losses.MeanAbsoluteError()
        ingredient_multilabel_loss = keras.losses.BinaryCrossentropy()
        category_classification_metrics = (
            [
                keras.metrics.CategoricalAccuracy(name="acc"),
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
            ],
        )

        calorie_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        carbs_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        protein_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        fat_regression_metrics = [keras.metrics.MeanAbsoluteError(name="MAE")]
        ingredient_multilabel_metrics = (
            [
                keras.metrics.Precision(name="precision"),
                keras.metrics.Recall(name="recall"),
            ],
        )
        category_classification_loss_weights = 1.0
        ingredient_multilabel_loss_weights = 1.0
        calorie_regression_loss_weights = 1.0
        carbs_regression_loss_weights = 1.0
        protein_regression_loss_weights = 1.0
        fat_regression_loss_weights = 1.0

        self.model.compile(
            optimizer=optimizer,
            loss={
                "category_output": category_classification_loss,
                "calorie_output": calorie_regression_loss,
                "carbs_output": carbs_regression_loss,
                "protein_output": protein_regression_loss,
                "fat_output": fat_regression_loss,
                "ingredients_output": ingredient_multilabel_loss,
            },
            metrics={
                "category_output": category_classification_metrics,
                "calorie_output": calorie_regression_metrics,
                "carbs_output": carbs_regression_metrics,
                "protein_output": protein_regression_metrics,
                "fat_output": fat_regression_metrics,
                "ingredients_output": ingredient_multilabel_metrics,
            },
            loss_weights={
                "category_output": category_classification_loss_weights,
                "calorie_output": calorie_regression_loss_weights,
                "carbs_output": carbs_regression_loss_weights,
                "protein_output": protein_regression_loss_weights,
                "fat_output": fat_regression_loss_weights,
                "ingredients_output": ingredient_multilabel_loss_weights,
            },
        )

In [ ]:
def freeze_category_classification_layers(self):
    assert self.model is not None
    submodel = self.model.get_layer("category_output")
    submodel.trainable = False


In [ ]:
def freeze_convolution_base(self):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetB1")
        submodel.trainable = False

In [ ]:
def get_callbacks(self):
        tensorboard_dir = f"./models/logs/{self.name}/{self.model_config_name}"
        checkpoint_dir = f"./temp/checkpoint/{self.name}/{self.model_config_name}"
        checkpoint_model_path = checkpoint_dir + "/model"
        checkpoint_weights_path = checkpoint_dir + "/weights/ckpt"
        checkpoint_model_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_model_path, monitor="val_loss", save_best_only=True, mode="min"
        )
        checkpoint_weights_callback = keras.callbacks.ModelCheckpoint(
            checkpoint_weights_path,
            monitor="val_loss",
            save_best_only=True,
            save_weights_only=True,
            mode="min",
        )
        reduce_lr_callback = keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss", factor=0.5, patience=2, min_lr=1e-6, verbose=1
        )
        tensorboard_callback = keras.callbacks.TensorBoard(log_dir=tensorboard_dir)
        early_stopping_callback = keras.callbacks.EarlyStopping("val_loss", patience=8)
        return [
            tensorboard_callback,
            early_stopping_callback,
            checkpoint_model_callback,
            checkpoint_weights_callback,
            reduce_lr_callback,
        ]

In [ ]:
    # Layers from bottom to top (classifier)  #
    keras.Input(shape=self.input_shape)

    input_layer = keras.layers.Input(shape=self.input_shape)
    augmentation_layer = keras.layers.RandomFlip()(input_layer)
    augmentation_layer = keras.layers.RandomRotation(0.2)(augmentation_layer)
    keras.Model(inputs=input_layer, outputs=augmentation_layer, name="augmentation_layers")

        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        shared_layer = keras.layers.Flatten()(input_layer)
        shared_layer = keras.layers.Dense(
            num_units[0], activation="relu", name="shared_dense_1"
        )(shared_layer)
        shared_layer = keras.layers.BatchNormalization()(shared_layer)
        output_layer = keras.layers.Dropout(0.2)(shared_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="shared_layers"
        )

        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(
            num_units[0], activation="relu", name="category_dense_1"
        )(input_layer)
        x = keras.layers.BatchNormalization()(x)
        category_classification_layer = keras.layers.Dense(
            total_categories, activation="softmax", name="category_output_layer"
        )(x)
        output_model = keras.Model(
            inputs=input_layer,
            outputs=category_classification_layer,
            name="category_output",
        )
    
#     calorie layers
    input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="calorie_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="calorie_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        calorie_regression_layers = keras.layers.Dense(1, name="calorie_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=calorie_regression_layers, name="calorie_output"
        )

#     carbs layrers
#     def get_carbs_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="carbs_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="carbs_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        carbs_regression_layers = keras.layers.Dense(1, name="carbs_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=carbs_regression_layers, name="carbs_output"
        )

#     protein layers
#     def get_protein_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="protein_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="protein_dense_2")(
            x
        )
        x = keras.layers.BatchNormalization()(x)
        protein_regression_layers = keras.layers.Dense(1, name="protein_output_layer")(
            x
        )
        return keras.Model(
            inputs=input_layer, outputs=protein_regression_layers, name="protein_output"
        )

    
# fat layers
#     def get_fat_regression_layers(self, input_tensor, *num_units):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        x = keras.layers.Dense(num_units[0], activation="relu", name="fat_dense_1")(
            input_layer
        )
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(num_units[1], activation="relu", name="fat_dense_2")(x)
        x = keras.layers.BatchNormalization()(x)
        fat_regression_layers = keras.layers.Dense(1, name="fat_output_layer")(x)
        return keras.Model(
            inputs=input_layer, outputs=fat_regression_layers, name="fat_output"
        )

#     ingre
#     def get_ingredients_multilabel_layers(
#         self, input_tensor, total_ingredients, *num_units
#     ):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[0], activation="relu", name="ingredients_dense_1"
        )(input_layer)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[1], activation="relu", name="ingredients_dense_2"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        ingredients_multilabel_layers = keras.layers.Dense(
            num_units[2], activation="relu", name="ingredients_dense_3"
        )(ingredients_multilabel_layers)
        ingredients_multilabel_layers = keras.layers.BatchNormalization()(
            ingredients_multilabel_layers
        )
        output_layers = keras.layers.Dense(
            total_ingredients, activation="sigmoid", name="ingredients_output_layer"
        )(ingredients_multilabel_layers)
        return keras.Model(
            inputs=input_layer, outputs=output_layers, name="ingredients_output"
        )

    # End of layers  #

    def load_model(self, path):
        self.model = keras.models.load_model(path)
        
    def evaluate(self, validation_dataset):
        assert self.model is not None
        self.model.evaluate(validation_dataset)  
    
    def predict(self, validation_dataset):
        assert self.model is not None
        self.model.predict(validation_dataset)

    def print_summary(self):
        assert (
            self.model is not None
        ), "Please run build_and_compile before printing summary."
        self.model.summary(expand_nested=True, show_trainable=True)

    def save_model(self, path):
        assert self.model is not None
        self.model.save(path, save_format="h5")

    def train_model(self, **kwargs):
        assert self.model is not None, "No model found."
        return self.model.fit(**kwargs, callbacks=self.get_callbacks())

    def unfreeze_category_classification_layers(self):
        assert self.model is not None
        submodel = self.model.get_layer("category_output")
        submodel.trainable = False

    def unfreeze_convolution_base(self, fine_tune_at=0):
        assert self.model is not None
        submodel = self.model.get_layer("efficientnetb1")
        submodel.trainable = True
        for layer in submodel.layers[:fine_tune_at]:
            layer.trainable = False


In [ ]:
class FlatModel(BaseModel):
        model_inputs = self.get_input_layer()
        prev_layer = self.get_augmentation_layers()(model_inputs)
        prev_layer = self.get_preprocess_layers(prev_layer)(prev_layer)
        prev_layer = self.get_convolution_block()(prev_layer)
        prev_layer = self.get_shared_layers(prev_layer, *shared_units)(prev_layer)
        category_classification_head = self.get_category_classification_layers(
            prev_layer, self.total_food_category, *independent_category_units
        )(prev_layer)
        ingredients_multilabel_head = self.get_ingredients_multilabel_layers(
            prev_layer, self.total_ingredients_category, *independent_ingredients_units
        )(prev_layer)
        calorie_regression_head = self.get_calorie_regression_layers(
            prev_layer, *independent_calorie_units
        )(prev_layer)
        carbs_regression_head = self.get_carbs_regression_layers(
            prev_layer, *independent_carbs_units
        )(prev_layer)
        protein_regression_head = self.get_protein_regression_layers(
            prev_layer, *independent_protein_units
        )(prev_layer)
        fat_regression_head = self.get_fat_regression_layers(
            prev_layer, *independent_fat_units
        )(prev_layer)

        model = keras.Model(
            inputs=model_inputs,
            outputs=[
                category_classification_head,
                ingredients_multilabel_head,
                calorie_regression_head,
                carbs_regression_head,
                protein_regression_head,
                fat_regression_head,
            ],
            name=self.name,
        )
        self.model = model


In [ ]:
class FlatEfficientNetB1Model(FlatModel):
    def get_preprocess_layers(self, input_tensor):
        input_layer = keras.layers.Input(shape=input_tensor.shape[1:])
        output_layer = keras.applications.efficientnet.preprocess_input(input_layer)
        return keras.Model(
            inputs=input_layer, outputs=output_layer, name="preprocessing_layers"
        )

    def get_convolution_block(self):
        efficientnet_convolution_layers = (
            keras.applications.efficientnet.EfficientNetB1(
                input_shape=self.input_shape,
                include_top=False,
                weights="imagenet",
                pooling="avg",
            )
        )
        efficientnet_convolution_layers.trainable = False
        return efficientnet_convolution_layers


In [ ]:
flat_efficientnet = FlatEfficientNetB1Model(
    input_shape=(224, 224, 3),
    total_food_category=len(EXPORTED.one_hot_encoder.all_food_categories),
    total_ingredients_category=len(EXPORTED.one_hot_encoder.all_ingredients),
    model_config_name="finalIngredients",
)

flat_efficientnet.build(
    shared_units=[2048],
    independent_category_units=[512],
    independent_ingredients_units=[1024, 512, 256],
    independent_protein_units=[64, 32],
    independent_fat_units=[64, 32],
    independent_carbs_units=[64, 32],
    independent_calorie_units=[64, 32],
)


In [ ]:
model.fit(x=train_dataset, validation_data=validation_dataset, epochs=30, verbose=2)

In [ ]:
model.evaluate(validation_dataset)

In [ ]:
model.save('/kaggle/working/model_100_epcohs', save_format="h5")

In [ ]:
image = tf.keras.preprocessing.image.load_img('/kaggle/input/pizza-test/1001116.jpg')
img_tensor = tf.keras.preprocessing.image.img_to_array(image, dtype="uint8")
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor = tf.image.resize(img_tensor, (224, 224))
tf.cast(img_tensor, tf.uint8)

In [ ]:
category_output, ing_output, cal_output, carbs_output, pro_output, fat_output = model.predict(img_tensor)

In [ ]:
n = category_output[0]
cat_pred = np.where(n == np.max(n))[0][0]
print(cat_pred)
# 1 prediction -> 98 categories

for key, value in EXPORTED.one_hot_encoder.all_food_categories_integer_encoded.items():
    if value == cat_pred:
        print("Category:", key)

i = ing_output[0]
l = np.where(i >= 0.5)
for key, value in EXPORTED.one_hot_encoder.all_ingredients_integer_encoded.items():
    for ing_pred in l[0]:
        if value == ing_pred:
            print("Ingredients", key)
# prediction -> 871 ingredients

print("Calories:", cal_output[0])
# 1 prediction -> 1 cal

print("Carbs:", carbs_output[0])
# 1 prediction -> 1 carbs

print("Proteins:", pro_output[0])
# 1 prediction -> 1 protein

print("Fat:", fat_output[0])
# 1 prediction -> 1 fat

In [ ]:
# converting .h5 model to .json
!tensorflowjs_converter --input_format=keras '/kaggle/input/model-100-epochs/model_100_epcohs' '/kaggle/working/'